In [1]:
import pyodbc
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
from pmdarima.arima import auto_arima
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.models import HoverTool, FactorRange, Plot, LinearAxis, Grid,Range1d,DatetimeTickFormatter
from bokeh.models.widgets import Panel,Tabs
from bokeh.models.glyphs import VBar
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.models.sources import ColumnDataSource
from bokeh.io import show


In [2]:
server = '(localdb)\MSSQLLocalDB'
database = 'SCMdb'
username = 'Guest'
password = 'Guest'
driver= '{ODBC Driver 17 for SQL Server}'
con = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+database+";Uid="+username+";Pwd="+password+";TrustServerCertificate=no;Connection Timeout=30;")



In [3]:
SQL_Query = pd.read_sql_query("select * from show_sales where prod_id='pr1'", con)
df=pd.DataFrame(SQL_Query)
df['date'] = pd.to_datetime(df['date'])
df=df.drop(['prod_id'], axis=1)
df=df.set_index('date')



In [4]:
df.head()

,Quantity
date,
2013-07-02,17
2013-07-06,16
2013-07-07,34
2013-07-13,22
2013-07-17,16


In [5]:
y=df
y = df['Quantity'].resample('MS').sum()

train=y[0:-2]
valid=y[-12:]


date
2018-12-01     533
2019-01-01    2328
2019-02-01    2428
2019-03-01    2800
2019-04-01    2100
2019-05-01    2100
2019-06-01    2210
2019-07-01    2320
2019-08-01    2420
2019-09-01    2526
2019-10-01    2400
2019-11-01    2800
Freq: MS, Name: Quantity, dtype: int64

In [197]:

model = auto_arima(y, trace=True, start_p=3, start_q=3, start_P=1, start_Q=5,
                     max_p=7, max_q=7, max_P=7, max_order=20,max_Q=6,D=1,d=1, m=1,seasonal=True,
                     stepwise=True, error_action='ignore', suppress_warnings=True)
model.fit(y)
forecast = model.predict(n_periods=4)


Fit ARIMA: order=(3, 1, 3) seasonal_order=(0, 0, 0, 1); AIC=1074.291, BIC=1093.042, Fit time=0.468 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=1074.382, BIC=1079.070, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=1075.954, BIC=1082.986, Fit time=0.103 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=1076.031, BIC=1083.063, Fit time=0.096 seconds
Fit ARIMA: order=(2, 1, 3) seasonal_order=(0, 0, 0, 1); AIC=1072.344, BIC=1088.750, Fit time=0.558 seconds
Fit ARIMA: order=(2, 1, 2) seasonal_order=(0, 0, 0, 1); AIC=1074.215, BIC=1088.278, Fit time=0.605 seconds
Fit ARIMA: order=(2, 1, 4) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 1); AIC=1074.246, BIC=1085.965, Fit time=0.361 seconds
Fit ARIMA: order=(3, 1, 4) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 3) seasonal_order=(0, 0, 0, 1); AIC=1

In [198]:
date_index=y[-1:].index


In [199]:
x_index=[]
date_index=date_index+1
for i in range(4):
    x_index.append(str((date_index+i).date[0]))

In [200]:
x_index

['2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01']

In [201]:
forecast

array([2760.58341911, 2896.05178695, 2778.23223667, 2811.49034756])

In [230]:
forecast=forecast.round()
forecast=forecast.astype(int)

In [231]:
d = {'Date': x_index, 'Prediction': forecast}
result = pd.DataFrame(data=d)
#forecast = pd.DataFrame(forecast,index = x_index,columns=['Prediction'])
#forecast.to_csv('forecast.csv')



In [232]:
result.insert(loc=0, column='prod_id', value='pr1')

In [233]:
result

,prod_id,Date,Prediction
0,pr1,2019-12-01,2761
1,pr1,2020-01-01,2896
2,pr1,2020-02-01,2778
3,pr1,2020-03-01,2811


In [243]:
cur = con.cursor()
for index,row in result.iterrows():
    cur.execute("exec prc_insertForecast @prod_id=?,@fdate=?,@no=?",row['prod_id'],row['Date'],row["Prediction"])
cur.commit()
cur.close()

In [170]:
forecast.head()

,Prediction
2019-12-01,2761.0
2020-01-01,2896.0
2020-02-01,2778.0
2020-03-01,2811.0


In [171]:
hover1 = HoverTool(tooltips=[("Sales", "@y")])
X = x_index
Y = forecast['Prediction'].iloc[-4:].values

p = figure(x_range=X, plot_height=400,x_axis_label='Month',x_minor_ticks=1,
         title="Forecast Sales",toolbar_location=None, tools=[hover1])
#barchart.vbar(x=X, top=Y, width=0.2)
p.line(y=Y,x=X,line_width=2)

p.y_range.start =2000
show(p)